In [ ]:
#r "nuget: BserClient, 1.1.1"

In [ ]:
using System.Text.Json;
using System.Text.Json.Serialization;
using System.IO;
using BserClient;
using System.Collections.Generic;

string readText = File.ReadAllText("games.json");

List<BserClient.Types.UserGameObj> games = 
JsonSerializer.Deserialize<List<BserClient.Types.UserGameObj>>(readText);

// List<dynamic> games = JsonSerializer.Deserialize<List<dynamic>>(readText);
// var games = Json.Decode(readText);
int ohioGames = 0;
int koreaGames = 0;
foreach (var game in games)
{
    if (game.matchingMode == 3 && game.seasonId == 3) {
        if (game.serverName == "Ohio") ohioGames += 1;
        if (game.serverName == "Seoul")  koreaGames += 1;
    }
}

Console.WriteLine(ohioGames);
Console.WriteLine(koreaGames);

431


315
